In [26]:
import pandas as pd
import pprint as pp
import numpy as np
import streamlit as st
import requests
from requests import post, get
import base64
import urllib
import os
import json
from dotenv import load_dotenv
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px

In [2]:
df= pd.read_excel(r'C:\Users\sofia\OneDrive\Documentos\GitHub\bigdataproject\app\data\country-available-final.xlsx')
df.head(2)

,country,capital,cap_lat,cap_lon,country_code,continent
0,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
1,Nauru,Yaren,-0.547700,166.920867,NR,Australia


In [3]:
# Spotify Developer Dashboard details
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# We first request a token to the Spotify Accounts Service, which will be used later on to access the Spotify Web API
# To get our access token, we need to pass our client ID, client Secret and grant_type
def get_token():
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode("utf-8")
    # Spotify requests to encode the client ID and client secret using base64
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    # Make the request
    result = post(url,headers= headers, data= data)
    # Convert the result (json) into a python dictionary
    json_result = json.loads(result.content) 
    token = json_result["access_token"]
    return token

In [4]:
token= get_token()

In [5]:
# Function to construct the header to send a request
def get_auth_header(token):
    return{"Authorization": "Bearer " + token}

In [6]:
# Get new releases
def get_new_releases(country):
    # Find the country code for the given country name
    row = df[df['country'] == country]
    country_code = row.iloc[0]['country_code']
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country_code}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)['albums']['items'][0]['name']
    return json_result

In [13]:
# Get new releases artist
def get_artist_new_releases(country):
    # Find the country code for the given country name
    row = df[df['country'] == country]
    country_code = row.iloc[0]['country_code']
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country_code}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)['albums']['items'][0]['artists'][0]['name']
    return json_result

In [14]:
get_artist_new_releases('Argentina')

'Barbi Recanati'

In [44]:
# Get the new release ID, so that later we can search for the tracks
def get_new_releases_id(country):
    # Find the country code for the given country name
    row = df[df['country'] == country]
    country_code = row.iloc[0]['country_code']
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country_code}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)['albums']['items'][0]['id']
    return json_result

In [45]:
# With the ID, get the tracks of that album
def tracks_from_album(country):
    album_id= get_new_releases_id(country)
    url= f"https://api.spotify.com/v1/albums/{album_id}/tracks?limit=15"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    for item in json_result['items']:
        name=item['name']
        number= item['track_number']
        print(number,name)
    #return print(number,name)

In [47]:
test= tracks_from_album('Spain')
test

1 MIENTRAS ME CURO DEL CORA
2 X SI VOLVEMOS
3 PERO TÚ
4 BESTIES
5 GUCCI LOS PAÑOS
6 TQG
7 TUS GAFITAS
8 OJOS FERRARI
9 MERCURIO
10 GATÚBELA
11 KÁRMIKA
12 PROVENZA
13 CAROLINA
14 DAÑAMOS LA AMISTAD
15 AMARGURA


# Map

In [7]:
df.iloc[115,4]

nan

In [8]:
df.iloc[115,4]='NA'

In [9]:
df.iloc[115,:]

country           Namibia
capital          Windhoek
cap_lat        -22.566667
cap_lon         17.083333
country_code           NA
continent          Africa
Name: 115, dtype: object

In [10]:
df['new_releases']= [get_new_releases(df.loc[index,'country']) for index in range(len(df))]

In [16]:
df['artist']= [get_artist_new_releases(df.loc[index,'country']) for index in range(len(df))]

In [11]:
df.head()

,country,capital,cap_lat,cap_lon,country_code,continent,new_releases
0,Palestine,Jerusalem,31.766667,35.233333,PS,Asia,TUYA
1,Nauru,Yaren,-0.547700,166.920867,NR,Australia,BLONDE16
2,Albania,Tirana,41.316667,19.816667,AL,Europe,Ku Je Ti (feat. Ricky Rich & Dafina Zeqiri)
3,Algeria,Algiers,36.750000,3.050000,DZ,Africa,YA LE3DOUWA
4,Andorra,Andorra la Vella,42.500000,1.516667,AD,Europe,Drapeau Blanc


In [29]:
fig = px.scatter_geo(df, lat='cap_lat', lon= 'cap_lon',
                     hover_name='new_releases', color='country',
                     hover_data={'cap_lat': False, # Don't show latitude and longitud
                                 'cap_lon': False,
                                 'artist': True,   # Show artist and country
                                 'country': True}
                     ) 

# Update the layout with the desired colors
'''fig.update_layout(
    geo=dict(
        bgcolor='black',  # Set the background color to black
        showland=True,
        landcolor='white',  # Set the color of land areas to white
        showocean=True,
        oceancolor='black',  # Set the color of ocean areas to black
        showlakes=True,
        lakecolor='white'  # Set the color of lakes to white
    ),
    font=dict(
        color='white'  # Set the text color to white
    )
)'''
fig.show()

In [27]:
#test = df.groupby('continent')['continent'].count()
fig2 = px.scatter_mapbox(df, lat='cap_lat', lon='cap_lon', title="Number of countries")
pyo.iplot(fig2)